In [7]:
import zipfile

with zipfile.ZipFile("/content/script.zip", 'r') as zip_ref:
    zip_ref.extractall("/content/script")  # Giải nén vào thư mục này

In [10]:
import os
import json
import pandas as pd

script_folder = "/content/script/script/L01"  # <- cập nhật đúng path
all_segments = []

for filename in os.listdir(script_folder):
    if filename.endswith("_script.json"):
        video_id = filename.replace("_script.json", "")
        file_path = os.path.join(script_folder, filename)

        with open(file_path, "r", encoding="utf-8") as f:
            try:
                data = json.load(f)
                if isinstance(data, list):
                    for seg in data:
                        if all(k in seg for k in ["start", "end", "text"]):
                            all_segments.append({
                                "video": video_id,
                                "start": seg["start"],
                                "end": seg["end"],
                                "text": seg["text"]
                            })
            except Exception as e:
                print(f"❌ Lỗi đọc {filename}: {e}")

# Tạo DataFrame
df = pd.DataFrame(all_segments)
print(f"✅ Đã load {len(df)} đoạn từ {df['video'].nunique()} video.")
df.head()

✅ Đã load 941 đoạn từ 8 video.


,video,start,end,text
0,L01_V007,0.00,0.64,Cái chào quý khán giả Hãy theo dõi chương trìn...
1,L01_V007,31.80,32.52,"Đêm qua, lửa bao trừng nhà hàng trong khu công..."
2,L01_V007,32.56,34.36,Và ngay sau đây là phần nội dung chi tiết. Tối...
3,L01_V007,35.64,37.16,"Theo đó, công tác chấm thi lớp 10 sẽ kết thúc ..."
4,L01_V007,55.32,57.32,"Từ ngày 18 đến ngày 20 tháng 6, hội phách đối ..."


In [11]:
!pip install -q faiss-cpu sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 60.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 34.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 87.3 MB/s eta 0:00:00


In [35]:
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss

model1 = SentenceTransformer("all-MiniLM-L6-v2")

# Encode tất cả đoạn text
embeddings1 = model1.encode(df["text"].tolist(), show_progress_bar=True)

# Tạo FAISS index có thể truy ID ngược lại
index1 = faiss.IndexIDMap(faiss.IndexFlatL2(embeddings1.shape[1]))
index1.add_with_ids(np.array(embeddings1), np.arange(len(df)))

Batches:   0%|          | 0/30 [00:00<?, ?it/s]

In [32]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')
# Encode tất cả đoạn text
embeddings = model.encode(df["text"].tolist(), show_progress_bar=True)

# Tạo FAISS index có thể truy ID ngược lại
index = faiss.IndexIDMap(faiss.IndexFlatL2(embeddings.shape[1]))
index.add_with_ids(np.array(embeddings), np.arange(len(df)))


Batches:   0%|          | 0/30 [00:00<?, ?it/s]

In [75]:
# all-MiniLM-L6-v2
def search_transcript1(query, top_k=5):
    query_vec = model1.encode([query])
    D, I = index1.search(np.array(query_vec), top_k)

    print(f"\n🔍 Kết quả cho: “{query}”\n")
    for idx in I[0]:
        row = df.iloc[idx]
        print(f"- 📺 {row['video']} | 🕒 {row['start']:.2f}s → {row['end']:.2f}s")
        print(f"  📜 {row['text']}\n")

# Ví dụ tìm kiếm
search_transcript1("cuộc thi sắc đẹp hoa hậu")


🔍 Kết quả cho: “cuộc thi sắc đẹp hoa hậu”

- 📺 L01_V002 | 🕒 199.40s → 201.08s
  📜 Mối đô hè về thì cũng là lúc nỗi lo đu nước âm ảy trở thành hiểm hòa hiền hữu, đặc biệt ở vùng sâu, vùng xa, vùng đồng bào dân tộc thiểu số.

- 📺 L01_V006 | 🕒 932.20s → 934.77s
  📜 Từ đây, trái cây Việt không chỉ có trên bàn ăn mà còn mang dấu ứng văn hóa, nghệ thuật đặc sắc cũng là hồn cốt dân tộc.

- 📺 L01_V006 | 🕒 722.23s → 726.53s
  📜 và phải dám đi đến tận cùng của con đường này.

- 📺 L01_V007 | 🕒 1114.12s → 1116.20s
  📜 Mỗi tấm huy chương sẽ được khắc tên của người dành chiến thắng và nội dung họ thi đấu.

- 📺 L01_V008 | 🕒 888.37s → 893.57s
  📜 Loại hoa này được dùng làm nước hoa hay lưu cưng, một loại bánh kẹo truyền thống vùng địa Trung Hải. Thị trấn Peme có 7.000 cư dân, hoa hồng như một nghi lễ.



In [74]:
# sentence-transformers/paraphrase-multilingual-mpnet-base-v2
def search_transcript(query, top_k=5):
    query_vec = model.encode([query])
    D, I = index.search(np.array(query_vec), top_k)

    print(f"\n🔍 Kết quả cho: “{query}”\n")
    for idx in I[0]:
        row = df.iloc[idx]
        print(f"- 📺 {row['video']} | 🕒 {row['start']:.2f}s → {row['end']:.2f}s")
        print(f"  📜 {row['text']}\n")

# Ví dụ tìm kiếm
search_transcript("cuộc thi sắc đẹp hoa hậu")


🔍 Kết quả cho: “cuộc thi sắc đẹp hoa hậu”

- 📺 L01_V005 | 🕒 856.40s → 875.00s
  📜 Cụ thể, tổ chức của Thái Lan đã trao danh hiệu á hậu 1 cho đại diện Brazil Talia Hartman, người gian á hậu 4 trước đó, danh hiệu á hậu 2 cho người đẹp Anh Quốc, á hậu 3 là Hoa hậu Tây Ban Nha, con vị trí á hậu 4 được trao cho người đẹp Cộng hòa Dominicana, hai cô gái trước đó chỉ có tên trong top 10.

- 📺 L01_V005 | 🕒 835.33s → 856.37s
  📜 Lần đầu tiên trong lịch sử, một trong những cuộc thi sát đẹp quy mô nhất thế giới có sự xáo trộn đáng kể về thứ hạn sau khi lượng lược khoa hậu và các á hậu tuyên bố từ bỏ danh hiệu. Tổ chức Israel International Hoa hậu Hòa Bình Quốc tế vừa có thông báo về các thứ hạn chính thức của lần tổ chức mới nhất 8 tháng sau khi đêm chung kết khép lại.

- 📺 L01_V006 | 🕒 133.93s → 135.33s
  📜 là khách mời danh dự của diễn đàn.

- 📺 L01_V005 | 🕒 929.57s → 930.43s
  📜 Không chỉ đua thuyền, giải đấu năm nay mang đến hình ảnh độc đáo với cuộc thi hóa trang.

- 📺 L01_V008 | 🕒 739.27s 